In [ ]:
import os
os.getcwd()

'/home/jovyan/activities_data/hi__paris_2022_hackathon/final_challenge/group_00099_shared_workspace'

In [30]:
import torch
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import glob

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /home/jovyan/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-3-6 torch 1.10.0+cu113 CUDA:0 (Tesla V100S-PCIE-32GB, 32510MiB)

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [28]:
from keras.models import load_model
import cv2
model_emissions = load_model('/home/jovyan/activities_data/hi__paris_2022_hackathon/final_challenge/group_00099_shared_workspace/models/model_pred_emission.h5')

In [29]:
target_size = (188,256,3)
im_path = glob.glob("../datasets/datasets_test/test/*")[0]
image = Image.open(im_path)
im = np.asanyarray(image)
im = cv2.resize(im, dsize=target_size[:2], interpolation=cv2.INTER_CUBIC)
plt.imshow(im)
im = np.expand_dims(im, axis=0)
im.shape

def make_prediction(model, im):
    classes_emissions = np.array([[129.76678445],
       [161.6751614 ],
       [199.10837695],
       [335.06065575]]).ravel()
    
    mean_emission = 201.73246813265277
    pred = model.predict(im)
    
    pred_emission = np.array([classes_emissions[i] for i in np.argmax(pred, axis =1)]).ravel()

    # pred_emission_ponderees = (pred_emission@classes_emissions).ravel() ##* 150/pred_emission.mean()
    # pred_emission_rescalees = pred_emission * mean_emission/pred_emission_ponderees.mean()
    
    return pred_emission[0]

# Test on one sample
make_prediction(model_emissions, im)

161.6751614

In [ ]:
crop = True

df = pd.DataFrame(columns = ["im_name","x_min","y_min","x_max","y_max", "e"])
for i, img in enumerate(glob.glob("../datasets/datasets_test/test/*")):
    row = [img.split("/")[-1]]
    im = np.asanyarray(Image.open(img).convert('RGB'))
    predictions = model(im).pandas().xyxy[0]
    predictions = predictions[((predictions["name"]=="car") | (predictions["name"]=="truck"))]
    try:
        running_box = predictions.loc[((predictions["xmax"]-predictions["xmin"]) * (predictions["ymax"] - predictions["ymin"])).argmax(), 
                               ["xmin","ymin","xmax","ymax"]].tolist()
        if crop:
            im_toPredict = np.asanyarray(Image.fromarray(im).crop([running_box[0],running_box[1],running_box[2],running_box[3]]))
            # print(im_toPredict.shape)
            im_toPredict = cv2.resize(im_toPredict, dsize=(256,188), interpolation=cv2.INTER_CUBIC)
            im_toPredict = np.expand_dims(im_toPredict.reshape((256,188,3)), axis=0)
            # pred_e.append(model_emissions.predict(im_toPredict))
            e = make_prediction(model_emissions, im_toPredict)
    except:
        running_box = [0,0,0,0]
        if crop:
            e = 0
    
    if crop == False:
        im_toPredict = cv2.resize(im, dsize=(256,188), interpolation=cv2.INTER_CUBIC)
        im_toPredict = np.expand_dims(im_toPredict.reshape((256,188,3)), axis=0)
        # pred_e.append(model_emissions.predict(im_toPredict))
        e = make_prediction(model_emissions, im_toPredict)
        
    row.extend(running_box)
    row.append(e)
    df.loc[i, :] = row
    # print(i)

df

In [ ]:
df.to_csv(
    "/home/jovyan/activities_data/hi__paris_2022_hackathon/final_challenge/group_00099_shared_workspace/submissions/submission.csv", index=False
)  # xxxx for your Hfactory ID group